<a href="https://colab.research.google.com/github/meliluc/marketing-analytics-customer-insights-project/blob/main/04_BI_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04 — Preparación de Tablas para BI (Power BI)

In [11]:
Objetivo:
- Generar tablas **dimensionales** (dim_date, dim_campaign, dim_customer)
- Generar tablas **de hechos** (fact_marketing, fact_churn)
- Guardar CSVs en `data/bi/` listos para importar en Power BI

Grano:
- `fact_marketing`: campaña–día
- `fact_churn`: cliente (Customer_ID sintético)

Relaciones esperadas en Power BI:
- fact_marketing.date_key → dim_date.date_key
- fact_marketing.Campaign_ID → dim_campaign.Campaign_ID
- fact_churn.Customer_ID → dim_customer.Customer_ID

SyntaxError: invalid character '–' (U+2013) (ipython-input-1271722906.py, line 7)

## 1) Setup + carga de datos limpios

In [42]:
import pandas as pd
from pathlib import Path
import numpy as np

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Rutas
BASE = Path("/content/drive/MyDrive/marketing-analytics-customer-insights")
CLEAN_DIR = BASE / "data" / "clean"
BI_DIR = BASE / "bi"
BI_DIR.mkdir(parents=True, exist_ok=True)

# Cargas base
df = pd.read_csv(CLEAN_DIR / "marketing_clean.csv", parse_dates=["Date"])

# Otras
rfm_df = pd.read_csv(CLEAN_DIR / 'rfm_clusters.csv')
rfm_path   = CLEAN_DIR / "rfm_clusters.csv"
churn_path = CLEAN_DIR / "churn_dataset.csv"
churn_df = pd.read_csv(churn_path) if churn_path.exists() else None

# ID sintético (replico igual que en 02_RFM/03_Churn)
df["Customer_ID"] = (df["Campaign_ID"] // 10).astype(int)

print("df shape:", df.shape)
print("rfm_df:", None if rfm_df is None else rfm_df.shape)
print("churn_df:", None if churn_df is None else churn_df.shape)

df.head(3)

Mounted at /content/drive
df shape: (200000, 23)
rfm_df: (20001, 5)
churn_df: (20001, 2)


,Campaign_ID,Company,Campaign_Type,Target_Audience,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROAS,Location,...,Engagement_Score,Customer_Segment,Date,CPA,CTR,CVR,CPC,CPM,ROI_pct,Customer_ID
0,1,innovate industries,email,men 18-24,30 days,google ads,0.04,16174.0,6.29,chicago,...,6,health & wellness,2021-01-01,16174.0,0.263267,0.04,31.964427,8415.192508,5.29,0
1,2,nexgen systems,email,women 35-44,60 days,google ads,0.12,11566.0,5.61,new york,...,7,fashionistas,2021-01-02,11566.0,0.015419,0.12,99.706897,1537.418583,4.61,0
2,3,alpha innovations,influencer,men 25-34,30 days,youtube,0.07,10200.0,7.18,los angeles,...,1,outdoor adventurers,2021-01-03,10200.0,0.075864,0.07,17.465753,1325.019486,6.18,0


## 2) Derivados previos en df (Duration_num)

In [43]:
# Convertir 'Duration' tipo "30 days" -> 30.0
df["Duration_num"] = (
    df["Duration"]
      .astype(str)
      .str.extract(r"(\d+)")
      .astype(int)
)

df[["Duration","Duration_num"]].head(8)


,Duration,Duration_num
0,30 days,30
1,60 days,60
2,30 days,30
3,60 days,60
4,15 days,15
5,15 days,15
6,60 days,60
7,45 days,45


## 3) dim_date (tabla calendario)

In [44]:
# Calendario entre min y max de Date
cal = pd.DataFrame({"Date": pd.date_range(df["Date"].min(), df["Date"].max(), freq="D")})
cal["date_key"]  = cal["Date"].dt.strftime("%Y%m%d").astype(int)
cal["year"]      = cal["Date"].dt.year
cal["quarter"]   = cal["Date"].dt.quarter
cal["month"]     = cal["Date"].dt.month
cal["month_name"]= cal["Date"].dt.strftime("%b")
cal["week"]      = cal["Date"].dt.isocalendar().week.astype(int)
cal["dow"]       = cal["Date"].dt.dayofweek
cal["dow_name"]  = cal["Date"].dt.day_name()

cal.to_csv(BI_DIR / "dim_date.csv", index=False)
print("✅ dim_date.csv →", (BI_DIR / "dim_date.csv"))

cal.head()


✅ dim_date.csv → /content/drive/MyDrive/marketing-analytics-customer-insights/bi/dim_date.csv


,Date,date_key,year,quarter,month,month_name,week,dow,dow_name
0,2021-01-01,20210101,2021,1,1,Jan,53,4,Friday
1,2021-01-02,20210102,2021,1,1,Jan,53,5,Saturday
2,2021-01-03,20210103,2021,1,1,Jan,53,6,Sunday
3,2021-01-04,20210104,2021,1,1,Jan,1,0,Monday
4,2021-01-05,20210105,2021,1,1,Jan,1,1,Tuesday


## 4) dim_campaign y algunas normalizaciones

In [45]:
# Partimos de un agregado único por Campaign_ID
dim_campaign = (
    df.groupby("Campaign_ID", as_index=False, observed=True)
      .agg({
          "Company":"first",
          "Campaign_Type":"first",
          "Target_Audience":"first",
          "Channel_Used":"first",
          "Location":"first",
          "Language":"first",
          "Customer_Segment":"first",
          "Duration":"first",
          "Duration_num":"first"
      })
)

# Normalizo Target_Audience en columnas separadas
# valores actuales: "men 18-24", "women 25-34", "all ages", separaremos género y rango de edad
ta = dim_campaign["Target_Audience"].astype(str).str.lower()

# Género: men / women / all / unknown
dim_campaign["Target_Gender"] = ta.str.extract(r"\b(men|women|all)\b", expand=False).fillna("unknown")

# Rango etario: "18-24", "25-34", etc. o 'all ages' / unknown
dim_campaign["Target_AgeRange"] = ta.str.extract(r"(\d{2}-\d{2}|all ages)", expand=False).fillna("unknown")

# Exportar
dim_campaign.to_csv(BI_DIR / "dim_campaign.csv", index=False)
print("✅ dim_campaign.csv →", (BI_DIR / "dim_campaign.csv"))
dim_campaign.head(10)



✅ dim_campaign.csv → /content/drive/MyDrive/marketing-analytics-customer-insights/bi/dim_campaign.csv


,Campaign_ID,Company,Campaign_Type,Target_Audience,Channel_Used,Location,Language,Customer_Segment,Duration,Duration_num,Target_Gender,Target_AgeRange
0,1,innovate industries,email,men 18-24,google ads,chicago,spanish,health & wellness,30 days,30,men,18-24
1,2,nexgen systems,email,women 35-44,google ads,new york,german,fashionistas,60 days,60,women,35-44
2,3,alpha innovations,influencer,men 25-34,youtube,los angeles,french,outdoor adventurers,30 days,30,men,25-34
3,4,datatech solutions,display,all ages,youtube,miami,mandarin,health & wellness,60 days,60,all,all ages
4,5,nexgen systems,email,men 25-34,youtube,los angeles,mandarin,health & wellness,15 days,15,men,25-34
5,6,datatech solutions,display,all ages,instagram,new york,german,foodies,15 days,15,all,all ages
6,7,nexgen systems,email,women 35-44,website,los angeles,spanish,tech enthusiasts,60 days,60,women,35-44
7,8,datatech solutions,search,men 18-24,google ads,los angeles,mandarin,outdoor adventurers,45 days,45,men,18-24
8,9,alpha innovations,social media,women 35-44,facebook,chicago,german,tech enthusiasts,15 days,15,women,35-44
9,10,techcorp,email,women 35-44,instagram,los angeles,english,tech enthusiasts,15 days,15,women,35-44


## 5) dim_customer (churn + rfm + KPIs de referencia)

In [46]:
# Base mínima: listado único de Customer_ID
dim_customer = df[["Customer_ID"]].drop_duplicates().copy()

# Añadir churn si existe
if churn_df is not None and {"Customer_ID","churn"} <= set(churn_df.columns):
    dim_customer = dim_customer.merge(
        churn_df[["Customer_ID","churn"]].drop_duplicates(),
        on="Customer_ID", how="left"
    )

# Añadir cluster RFM si existe
if rfm_df is not None and {"Customer_ID","Cluster"} <= set(rfm_df.columns):
    dim_customer = dim_customer.merge(
        rfm_df[["Customer_ID","Cluster"]].drop_duplicates(),
        on="Customer_ID", how="left"
    ).rename(columns={"Cluster":"rfm_cluster"})

# Métricas de referencia por cliente (útil para cortes y KPI cards en BI)
agg_ref = (
    df.groupby("Customer_ID", as_index=False, observed=True)
      .agg({
          "Acquisition_Cost":"sum",
          "Clicks":"sum",
          "Impressions":"sum",
          "CTR":"mean",
          "CVR":"mean",
          "ROAS":"mean"
      })
      .rename(columns={
          "Acquisition_Cost":"spend_total",
          "CTR":"ctr_mean",
          "CVR":"cvr_mean",
          "ROAS":"roas_mean"
      })
)
dim_customer = dim_customer.merge(agg_ref, on="Customer_ID", how="left")

# Exportar
dim_customer.to_csv(BI_DIR / "dim_customer.csv", index=False)
print("✅ dim_customer.csv →", (BI_DIR / "dim_customer.csv"))
dim_customer.head(10)



✅ dim_customer.csv → /content/drive/MyDrive/marketing-analytics-customer-insights/bi/dim_customer.csv


,Customer_ID,churn,rfm_cluster,spend_total,Clicks,Impressions,ctr_mean,cvr_mean,roas_mean
0,0,0,0,119245.0,4204,43164,0.143175,0.084444,5.625556
1,1,1,0,121962.0,5405,57982,0.150075,0.093000,4.611000
2,2,1,0,116833.0,6174,46747,0.207672,0.068000,5.616000
3,3,0,0,128647.0,5311,59838,0.097560,0.080000,4.273000
4,4,1,0,123372.0,5422,47147,0.172983,0.058000,5.191000
5,5,0,1,145298.0,3554,60780,0.082256,0.080000,5.136000
6,6,1,0,114166.0,5005,59118,0.093570,0.059000,5.688000
7,7,0,0,125010.0,5827,64043,0.112220,0.087000,5.351000
8,8,1,1,141514.0,4296,51942,0.113460,0.121000,5.245000
9,9,1,0,123390.0,5380,67292,0.088301,0.067000,5.006000


## 6) Tablas de hechos

### Fact_marketing (grano: Campaña–Día)

In [47]:
# 3) Generar tabla de hechos: fact_marketing
fact_marketing = (
    df.groupby(['Customer_ID', 'Campaign_ID', 'Date', 'Channel_Used'], as_index=False, observed=True)
      .agg({
          'Clicks': 'sum',
          'Impressions': 'sum',
          'Acquisition_Cost': 'sum',
          'CPC': 'mean',
          'CPM': 'mean',
          'CTR': 'mean',
          'CVR': 'mean',
          'ROAS': 'mean'
      })
      .rename(columns={'Acquisition_Cost': 'Spend'})
)

fact_marketing['date_key'] = fact_marketing['Date'].dt.strftime('%Y%m%d').astype(int)

# Exportar la tabla corregida a un nuevo CSV
fact_marketing.to_csv(BI_DIR / 'fact_marketing.csv', index=False)

print('✅ fact_marketing.csv actualizado con Customer_ID')
fact_marketing.head()


✅ fact_marketing.csv actualizado con Customer_ID


,Customer_ID,Campaign_ID,Date,Channel_Used,Clicks,Impressions,Spend,CPC,CPM,CTR,CVR,ROAS,date_key
0,0,1,2021-01-01,google ads,506,1922,16174.0,31.964427,8415.192508,0.263267,0.04,6.29,20210101
1,0,2,2021-01-02,google ads,116,7523,11566.0,99.706897,1537.418583,0.015419,0.12,5.61,20210102
2,0,3,2021-01-03,youtube,584,7698,10200.0,17.465753,1325.019486,0.075864,0.07,7.18,20210103
3,0,4,2021-01-04,youtube,217,1820,12724.0,58.635945,6991.208791,0.119231,0.11,5.55,20210104
4,0,5,2021-01-05,youtube,379,4201,16452.0,43.408971,3916.210426,0.090217,0.05,6.50,20210105


### fact_churn (grano: Cliente)

In [48]:
if churn_df is not None and {"Customer_ID","churn"} <= set(churn_df.columns):
    fact_churn = churn_df[["Customer_ID","churn"]].drop_duplicates().copy()
    fact_churn.to_csv(BI_DIR / "fact_churn.csv", index=False)
    print("✅ Exportado fact_churn.csv en: ", (BI_DIR / "fact_churn.csv"))
    fact_churn.head()
else:
    print("ℹ️ No se encontró churn_dataset.csv con ['Customer_ID','churn']. Se omite fact_churn.")


✅ Exportado fact_churn.csv en:  /content/drive/MyDrive/marketing-analytics-customer-insights/bi/fact_churn.csv


## 7) Tablas de RFM y Churn juntas

In [51]:
## Unimos datos de RFM y de abandono en una sola tabla para BI
# 1) Unir las tablas a nivel de cliente
customer_data = rfm_df.merge(churn_df, on='Customer_ID', how='inner')

# 2) Calcular el EngagementProxy (Clicks) y unirlo
engagement_proxy = df.groupby('Customer_ID').agg(
    Clicks=('Clicks', 'sum')
).reset_index()

customer_data = customer_data.merge(engagement_proxy, on='Customer_ID', how='left').fillna(0)

# 3) Normalizar EngagementProxy_norm_inv
min_clicks = customer_data['Clicks'].min()
max_clicks = customer_data['Clicks'].max()
customer_data['EngagementProxy_norm_inv'] = 1 - (customer_data['Clicks'] - min_clicks) / (max_clicks - min_clicks)

# 4) Normalizar Recency (Recency_norm)
min_recency = customer_data['Recency'].min()
max_recency = customer_data['Recency'].max()
customer_data['Recency_norm'] = (customer_data['Recency'] - min_recency) / (max_recency - min_recency)

# 5) Crear la columna Riesgo de Abandono
# Puedes ajustar los umbrales si lo necesitas.
customer_data['Riesgo de Abandono'] = np.where(
    (customer_data['Recency_norm'] > 0.4) &
    (customer_data['EngagementProxy_norm_inv'] > 0.4),
    'Alto',
    'Bajo'
)

# 6) Exportar la tabla final a un solo CSV
BI_DIR = Path("/content/drive/MyDrive/marketing-analytics-customer-insights/bi/")
customer_data.to_csv(BI_DIR / 'dim_customer_final.csv', index=False)
print("✅ dim_customer_final.csv ha sido creado.")

✅ dim_customer_final.csv ha sido creado.


## 7) Comprobaciones de archivos exportados necesarios

In [52]:
print("— Resumen de archivos BI —")
for name in ["dim_date.csv","dim_campaign.csv","dim_customer.csv","fact_marketing.csv","fact_churn.csv"]:
    p = BI_DIR / name
    print(f"{'✅' if p.exists() else '❌'} {name}", end="")
    if p.exists():
        try:
            tmp = pd.read_csv(p, nrows=5)
            print(f" | filas de muestra: {tmp.shape[0]} | columnas: {tmp.shape[1]}")
        except Exception as e:
            print(f" | (no se pudo leer muestra) {e}")
    else:
        print()

— Resumen de archivos BI —
✅ dim_date.csv | filas de muestra: 5 | columnas: 9
✅ dim_campaign.csv | filas de muestra: 5 | columnas: 12
✅ dim_customer.csv | filas de muestra: 5 | columnas: 9
✅ fact_marketing.csv | filas de muestra: 5 | columnas: 13
✅ fact_churn.csv | filas de muestra: 5 | columnas: 2


## 8) Notas y preparación para tablero Power BI

- Se construyeron tablas en formato **esquema estrella**:
  - Dimensiones: `dim_date`, `dim_campaign`, `dim_customer`.
  - Hechos: `fact_marketing`, `fact_churn`.
- Todas las tablas fueron exportadas como CSV a `data/bi/`.
- Están listas para cargarse en **Power BI** y modelar el dashboard ejecutivo de Marketing Analytics.

- **Archivos importados**: `data/bi/dim_date.csv`, `dim_campaign.csv`, `dim_customer.csv`, `fact_marketing.csv`, `fact_churn.csv` (si existe).
- **Relaciones**:
  - `fact_marketing[date_key]` → `dim_date[date_key]`
  - `fact_marketing[Campaign_ID]` → `dim_campaign[Campaign_ID]`
  - `fact_churn[Customer_ID]` → `dim_customer[Customer_ID]`
- **Slicers útiles**: `year`, `month_name`, `Channel_Used`, `Campaign_Type`, `Target_Gender`, `Target_AgeRange`, `rfm_cluster`, `churn`.
- **Ideas de páginas**:
  1. **Overview**: Spend, ROAS, CPA, CTR, CVR, churn rate.
  2. **Canales/Campañas**: ROAS y CPA por canal/campaña; serie temporal Spend vs ROAS.
  3. **RFM**: % por `rfm_cluster`, barras de Monetary/Recency medios por cluster.
  4. **Churn**: churn rate por canal/cluster; tabla de clientes (si corresponde).

👉 Próximo paso: en Power BI se definirán relaciones entre tablas y se crearán medidas DAX (ej. ROAS promedio, CPA medio, churn rate).
